<a href="https://colab.research.google.com/github/a07458666/TensorflowExercise/blob/master/Exercise4_Question.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [6]:
import tensorflow as tf
import os
import zipfile


DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if (logs.get('acc')> DESIRED_ACCURACY):
      print("\n正確率高達99.9%")
      self.model.stop_training = True
        
callbacks = myCallback()

--2019-06-02 07:40:15--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.198.128, 2607:f8b0:4001:c0a::80
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.198.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.02s   

2019-06-02 07:40:15 (125 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [0]:
# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc']
)

In [8]:
# This code block should create an instance of an ImageDataGenerator called train_datagen 
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
        '/tmp/h-or-s',
        target_size=(150,150),
        batch_size=10,
        class_mode='binary'
)

# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [9]:
model.summary()

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
train_happy_dir = os.path.join('/tmp/h-or-s/happy')
print('total training images:', len(os.listdir(train_happy_dir)))
train_asd_dir = os.path.join('/tmp/h-or-s/sad')
print('total training images:', len(os.listdir(train_asd_dir)))
#img = mpimg.imread()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 18496)             0         
__________

In [10]:
# This code block should call model.fit_generator and train for
# a number of epochs. 
history = model.fit_generator(
    train_generator,
    steps_per_epoch = 2,
    epochs=15,
    verbose=1,
    callbacks = [callbacks]
)
# Expected output: "Reached 99.9% accuracy so cancelling training!""

Epoch 1/15
8/8 [==============================] - 0s 34ms/step - loss: 1.8145 - acc: 0.4875
Epoch 2/15
8/8 [==============================] - 0s 43ms/step - loss: 0.4229 - acc: 0.8250
Epoch 3/15
8/8 [==============================] - 0s 26ms/step - loss: 0.1325 - acc: 0.9500
Epoch 4/15
8/8 [==============================] - 0s 35ms/step - loss: 0.2353 - acc: 0.9250
Epoch 5/15
8/8 [==============================] - 0s 26ms/step - loss: 0.0409 - acc: 0.9875
Epoch 6/15
8/8 [==============================] - 0s 36ms/step - loss: 0.0698 - acc: 0.9750
Epoch 7/15
8/8 [==============================] - 0s 26ms/step - loss: 0.0416 - acc: 0.9875
Epoch 8/15
7/8 [=========================>....] - ETA: 0s - loss: 0.0070 - acc: 1.0000
正確率高達99.9%
8/8 [==============================] - 0s 36ms/step - loss: 0.0084 - acc: 1.0000


In [0]:
import os, signal
os.kill(os.getpid(), signal.SIGKILL)